<a href="https://colab.research.google.com/github/vdantas40/Mini_proj_price_house/blob/main/Syst_Recompra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJETO DE RECOMENDAÇÃO E AUMENTO DE RECORRÊNCIA DE COMPRA

### 1.0 PACOTES

In [ ]:
import pandas_profiling
from ipywidgets import interact, widgets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import datetime as dt
from datetime import datetime
# LIBERAR VISÃO DO DRIVE NO DIRETÓRIO COLAB
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### USUARIOS E SEUS DIRETORIOS COLAB

In [ ]:
colabdirect = str(input('Digite seu nome: adriano; alexandre; gabriel; pedro; willian; yuri; vdc'))

if colabdirect == str:
  print('Seu nome')
elif colabdirect == 'alexandre':
  dfvendas = pd.read_csv('/content/drive/MyDrive/DNC/amostra_compras_dnc.csv')
elif colabdirect == "adriano":
  print('adriano')
elif colabdirect == "gabriel":
  dfvendas = pd.read_csv('/content/drive/MyDrive/Grupo17DNC/amostra_compras_dnc.csv')
  print('GABI')
elif colabdirect == 'willian':
  print('WILL')
elif colabdirect == 'pedro':
  print('pedro')
elif colabdirect == 'yuri':  
  print('yuri')
elif colabdirect == 'vdc':
  dfvendas = pd.read_csv('/content/drive/MyDrive/Grupo17DNC/amostra_compras_dnc.csv')

print(f'Dados do diretório do {colabdirect}'.format(colabdirect))
dfvendas.head(1)

Digite seu nome: adriano; alexandre; gabriel; pedro; willian; yuri; vdcVDC
Dados do diretório do VDC


,siteid,date,time_epoch_hr,visitor_visitorid,visitor_visitcount,sku,product_name,category,price,quantity,order_transactionid,order_revenue,current_purchase,total_purchases
0,SAP-5535001,2022-09-20,2022/09/20 21:59:43,0000f0c029a62613,2.0,400750,TRIUNFF LUGAR AMERICANO 45 CM X 30 CM,Acessorios/Mesa/Toalhas de Mesa e Jogos Americ...,11.28,4.0,1263134292244,219.67,1,1


In [ ]:
# SEM NECESSIDADE DE TRATAMENTO - sem dados nulos ou NAN
#msno.bar(dfvendas,figsize=(25,1))

In [ ]:
dfvendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929698 entries, 0 to 929697
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   siteid               929698 non-null  object 
 1   date                 929698 non-null  object 
 2   time_epoch_hr        929698 non-null  object 
 3   visitor_visitorid    929698 non-null  object 
 4   visitor_visitcount   929698 non-null  float64
 5   sku                  929698 non-null  object 
 6   product_name         929698 non-null  object 
 7   category             928279 non-null  object 
 8   price                929698 non-null  float64
 9   quantity             929698 non-null  float64
 10  order_transactionid  929698 non-null  object 
 11  order_revenue        929698 non-null  float64
 12  current_purchase     929698 non-null  int64  
 13  total_purchases      929698 non-null  int64  
dtypes: float64(4), int64(2), object(8)
memory usage: 99.3+ MB


### 2.0 TRATAMENTO DE DADOS ORIENTADO A RECOMPRA / RECOMENDAÇÃO

In [ ]:
# PRECISAMOS TRABALHAR COM CONTAGEM DE DATAS
dfvendas['time_epoch_hr'] = pd.to_datetime((dfvendas['time_epoch_hr']),format="%Y-%m-%d %H:%M")
# FILTRAR CLIENTE
#dfvendas['time_freq'] = dfvendas['time_epoch_hr'].diff()

# FILTRAR EMPRESA >>>>>>>>>>>>>>>>>>
dfsap5048 = dfvendas[(dfvendas['siteid'] =='SAP-5535048')]#&(dfvendas['category']== 'Agua Destilada - 210')]
# VAMOS RETIRAR ALGUMAS VARIÁVEIS PARA MELHORAR A VISÃO NO DF
dfsap5048.drop(columns = ['date','siteid','product_name','sku'], inplace = True)
# MOLDAR OBJETO PARA NUMÉRICO
dfsap5048['order_transactionid'] = dfsap5048['order_transactionid'].astype(int)
# IDENTIFICAR APENAS 1 LINHA POR ORDEM
dfsap5048 = dfsap5048.drop_duplicates(['order_transactionid'])
# MELHOR ORIENTAR A CADA CLIENTE 
##dfsap5048.sort_values(by='time_epoch_hr', ascending=True, inplace=True)
lista43 = dfsap5048['visitor_visitorid'].unique()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [277]:
## LISTA DE CLIENTES ÚNICA # 531 clientes únicos
xxx = dfsap5048[dfsap5048['total_purchases'] >=2]
l43 = xxx['visitor_visitorid'].unique()
print(len(l43),l43)

6522 ['0001030c8dc84ed2' '0003cb364618e8ca' '000ef5b554d08a2b' ...
 'fff05e608b07de30' 'fff6080665f0fe5c' 'ffff9f4d6845d28d']


In [ ]:
#lista = dfsap5048[dfsap5048['visitor_visitorid'] == x]
#lista = dfsap5048.groupby(by=x)['price'].sum()

In [278]:
# ESTUDAR A TX DE RECOMPRA: QUANTAS PESSOAS RECOMPRAM? PERCENTUAL QUE RECOMPRA? O QUE ELAS RECOMPRAM? QUAL O TEMPO QUE SE LEVA PARA RECOMPRA?
# CRUZAR INFORMAÇÕES SOBRE A RECORRÊNCIA DE COMPRA?
# QUAIS FEATURES MAIS IMPACTAM NA RECOMPRA (insigths)?
cliente    = []
quant      = []
valor      = []
timemean   = []
#tx_recomp  = []
timefim    = []
timeinicio = []

for i in l43:
  print(i)
  x = dfsap5048['visitor_visitorid'] == i

  c = len(dfsap5048[x])
  p = dfsap5048.groupby(by=x)['order_revenue'].sum() # type float
  #tx= dfsap5048.groupby(by=x)['total_purchases'].max()
  t = (dfsap5048.groupby(by=x)['time_epoch_hr'].diff()).sum()/c
  t1 = dfsap5048.groupby(by=x)['time_epoch_hr'].max()
  t0 = dfsap5048.groupby(by=x)['time_epoch_hr'].min()

  print(t)
  #if c > 5:
  cliente.append(i)
  quant.append(c)
  valor.append(p[1])
  #tx_recomp.append(tx)
  
  timemean.append(t)
  timefim.append(t1[1])
  timeinicio.append(t0[1])

listona = [cliente,quant],#valor,tmUser]

A saída de streaming foi truncada nas últimas 5000 linhas.
9e445817260b18ee
-25 days +16:38:36.666666667
9e4eb50de116e410
-19 days +16:04:40.500000
9e6a79d98285296e
-23 days +08:34:28.500000
9e7062d51600ced3
-1 days +21:03:25
9e7609112079de72
-34 days +19:40:12.500000
9e7729f51a593dc9
-1 days +22:45:54.200000
9e7ad3a5bc9c8631
-13 days +17:49:13.500000
9e7b28cec5c90146
22 days 05:33:43
9e7da1917d0f6c23
-36 days +17:59:15
9e7ef305749f1129
-26 days +00:05:49.666666667
9e800843e989e3cb
-23 days +04:12:46.500000
9e8d6bfcdaeae22c
-32 days +06:53:48
9e909784235ddadd
-37 days +19:36:16
9e95802ca8b77a30
-37 days +16:01:42
9eb951ae337b2a24
-39 days +03:05:09.500000
9ebfeaeba4d91520
7 days 04:59:33.142857142
9ecaa060940df9f3
-34 days +16:01:23
9ed1c542098bbd2c
-31 days +14:27:29.500000
9ed3286ccf9e7260
-15 days +11:50:52.200000
9ed86b9d47bdc36c
-38 days +15:51:46
9edec1a1a6dc48dd
-24 days +18:59:12.500000
9edecac7ada129af
5 days 11:11:44.666666666
9ee40c746db50617
-39 days +04:56:48
9ef20a6430a20

Ticket médio;
Custo de aquisição do cliente;
Lucratividade;
Taxa de conversão de vendas;
Índice de inadimplência;
Retorno sobre investimentos;
Satisfação do cliente.

In [279]:
# ESTUDAR A TX DE RECOMPRA: 
      #QUANTAS PESSOAS RECOMPRAM? total de pesssoas comprando 531
      # >>> Qual a taxa de recompra? 
      #QUAL O TEMPO QUE SE LEVA PARA RECOMPRA? O QUE ELAS RECOMPRAM?
      # >>> Quais os fatores influenciam e interferem na recompra
diasXX = pd.DataFrame(cliente)
diasXX['qt']        = quant # na média compram 6,9 ou 7x
diasXX['tick_mean']   = valor / diasXX['qt']  # order_revenue >>>>>>>>
diasXX['tx_rec']    = ((diasXX["qt"] / diasXX['qt'].sum())*100).map('{:,.2f}'.format) # não usar % '{:,.2f}%'

diasXX['timebuy']   = timemean
diasXX['first_buy'] = timeinicio
diasXX['last_buy']  = timefim
diasXX['dias_count']= diasXX['last_buy'] - diasXX['first_buy']
diasXX['dias_dist'] = diasXX['dias_count'] / diasXX['qt']
diasXX['tx_rec']    = diasXX['tx_rec'].astype(float) # para o agrup2 rode média
diasXX.head(3)

,0,qt,tick_mean,tx_rec,timebuy,first_buy,last_buy,dias_count,dias_dist
0,0001030c8dc84ed2,2,89.595000,0.01,42 days 00:06:29.500000,2022-08-22 15:07:47,2022-10-09 06:54:41,47 days 15:46:54,23 days 19:53:27
1,0003cb364618e8ca,2,26.225000,0.01,-39 days +06:04:07.500000,2022-04-29 16:36:46,2022-04-30 18:49:57,1 days 02:13:11,0 days 13:06:35.500000
2,000ef5b554d08a2b,6,171.166667,0.03,5 days 04:48:50.166666666,2022-05-05 22:14:18,2022-08-23 17:12:15,109 days 18:57:57,18 days 07:09:39.500000


In [ ]:
diasXX['qt'].sort_values().head(2)

103    7
87     7
Name: qt, dtype: int64

In [ ]:
'''
n = 3
l_qt = np.arange(diasXX['qt'].max()+1)
t = (len(np.arange(diasXX['qt'].max())))

splited = []
for i in range(n):
  start = (i*t/n)
  end = ((i+1)*t/n)
  splited.append(l[start:end])
print(splited)
'''

In [334]:
inicio = 0
f = 4
def chunks(lista, n):
    for i in range(inicio, len(lista), n):
        yield lista[i:i + n]

l_qt = np.arange(diasXX['qt'].max()+1)
#print(list(chunks(l_qt, f)))
grupo = (list(chunks(l_qt, f)))
grupo

[array([0, 1, 2, 3]),
 array([4, 5, 6, 7]),
 array([ 8,  9, 10, 11]),
 array([12, 13, 14, 15]),
 array([16, 17, 18, 19]),
 array([20, 21, 22, 23]),
 array([24, 25, 26, 27]),
 array([28, 29, 30, 31]),
 array([32, 33, 34, 35]),
 array([36, 37, 38, 39]),
 array([40, 41, 42, 43])]

In [365]:
# ESTUDAR A TX DE RECOMPRA: QUANTAS PESSOAS RECOMPRAM? PERCENTUAL QUE RECOMPRA? O QUE ELAS RECOMPRAM? QUAL O TEMPO QUE SE LEVA PARA RECOMPRA?
# CRUZAR INFORMAÇÕES SOBRE A RECORRÊNCIA DE COMPRA?
# QUAIS FEATURES MAIS IMPACTAM NA RECOMPRA (insigths)?
grupo_periodos = np.arange(0, diasXX['qt'].max(), 10 ) ## >>>> adequar com o for ▲▲▲▲▲
diasXX['grupo_rec'] = pd.cut(diasXX['qt'], grupo_periodos-1, duplicates='raise')

df_agrup1 = diasXX.groupby('grupo_rec', sort=True).sum()
df_agrup2 = diasXX.groupby('grupo_rec').agg({'tx_rec': [ 'min','mean','max']}) * 100
df_agrup3 = diasXX.groupby('grupo_rec', sort=True)['dias_count'].mean()
df_agrup4 = pd.concat([df_agrup1, df_agrup2,df_agrup3], axis=1)
df_agrup4['tx_rec'] = df_agrup4['tx_rec'].map("{:.2f}%".format)
df_agrup4

,qt,tick_mean,tx_rec,"(tx_rec, min)","(tx_rec, mean)","(tx_rec, max)",dias_count
grupo_rec,,,,,,,
"(-1, 9]",16754,1.416140e+06,91.42%,1.0,1.415609,5.0,47 days 18:18:06.030659647
"(9, 19]",649,1.483753e+04,3.73%,6.0,6.781818,10.0,132 days 05:06:07.436363636
"(19, 29]",160,7.625788e+03,0.90%,11.0,12.857143,15.0,119 days 08:06:48.571428572
"(29, 39]",34,1.607118e+03,0.19%,19.0,19.000000,19.0,4 days 07:26:11


In [ ]:
((diasXX['qt']-1).sum()) / diasXX['qt'].sum() # tx de recompra

0.8987479586281981

### USUÁRIO FULANO

In [ ]:
v = dfsap5048['category'].str.split(pat = '-', n = -1, expand = True) 
dfsap5048['category_split'] = v[0]

dfsap5048['category_split'].unique()


array(['Rolo para Esterilizacao ', 'Desinfectante ', 'Curativos ', ...,
       ' Esponja para Banho  ', 'Regua Endodontica ', 'Espelhos Diversos'],
      dtype=object)

###6.0 MODELO DE RECOMENDAÇÃO

In [ ]:
# DATASET SEGREGADOS PARA ANALISE TYPOS ISOLADAS
#dfnumber = dfsap5048.select_dtypes(include = 'number')
#dfescrita= dfsap5048.select_dtypes(include = 'object')
#dfdatas  = dfsap5048.select_dtypes(include = 'datetime')

In [ ]:
# A correlação de Pearson avalia a relação linear entre duas variáveis contínuas.
# Uma relação é linear quando a mudança em uma variável é associada a uma mudança proporcional na outra variável.
#dfnumber.corr(method='pearson')  # corr>> total_purchases com current purchase 82% # corr>> current com visitocount 40% # corr>> total current com visitcount 37%

In [ ]:
# A correlação de Spearman avalia a relação monotônica entre duas variáveis contínuas ou ordinais. 
# Em uma relação monotônica, as variáveis tendem a mudar juntas mas não necessariamente a uma taxa constante.
# O coeficiente de correlação de Spearman baseia-se nos valores classificados de cada variável, em vez de os dados brutos.
#dfnumber.corr(method='spearman') # corr>> total com current purchase 77% # corr>> visitorcount com current ou total 51% # corr>> quantity com price -63%

In [ ]:
#dfvendas[['date']].plot(figsize=(25,5))
dfvajuste = dfvendas.drop(columns = ['sku','product_name'])#, inplace = True])
(
  dfvajuste# == 'SAP-5535048']
 .groupby(dfvajuste.columns.tolist())#, dropna=False)
 .size()
 .to_frame('n_duplicates')
 .query('n_duplicates>0')
 .sort_values('n_duplicates', ascending=False)
)

n_duplicates
siteid      date       time_epoch_hr       visitor_visitorid visitor_visitcount category                                           price  quantity order_transactionid order_revenue current_purchase total_purchases              
SAP-5535048 2022-10-17 2022-10-17 10:47:59 a6cc1c22cdef8be6  1.0                Elastico                                           10.50  1.0      102691837           143044.72     1                1                          44
                                                                                                                                   6.90   1.0      102691837           143044.72     1                1                          31
                                                                                 Dente                                             15.90  2.0      102691837           143044.72     1                1                          31
            2022-10-14 2022-10-14 01:11:06 c1aae121c73d0fb3  1.0                Grampo para Isolamento                             24.90  1.0      102689397           4171.01       1                3                          28
            2022-10-17 2022-10-17 10:47:59 a6cc1c22cdef8be6  1.0                Elastico                                           15.75  1.0      102691837           143044.72     1                1                          27
...                                                                                                                                                                                                                             ...
SAP-5535001 2022-09-19 2022-09-19 16:11:46 793402ae65d39dfe  1.0                Acessorios/Decoracao/Vasos                         47.90  1.0      1262830695930       135.51        2                2                           1
                                                                                                                                   79.90  1.0      1262830695930       135.51        2                2                           1
                       2022-09-19 16:16:00 5f643a50e44d558e  5.0                Acessorios/Cozinha/Panos de Prato, Aventais e L... 35.90  1.0      1262830950439       227.77        1                1                           1
                                                                                Acessorios/Decoracao/Adornos e Objetos Decorativos 9.90   1.0      1262830950439       227.77        1                1                           1
SAP-5535058 2022-10-19 2022-10-19 11:46:11 084e73c2aaf4e54e  2.0                MASCULINO/MACACOES                                 259.90 1.0      1269990826154-01    259.90        1                1                           1

[875747 rows x 1 columns]

#### OUTROS

In [ ]:
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20,10))

counter = 0 

for i in range(nrows):
  for j in range(ncols):

    ax = axes[i][j]

    # Plotar somente quando houver dados
    if counter < len(contColNames):

      ax.hist(dfvendas[dfvendas[contColNames[counter]]<25].select_dtypes(include='number')[contColNames[counter]], bins=100)
      ax.set_xlabel(contColNames[counter])
      ax.set_ylabel('Frequência')

    else:
      ax.set_axis_off()

    counter += 1

plt.show()

NameError: ignored

### 7.0 GRÁFICOS para montar

In [ ]:
ds_temp = drugstore[['visitor_visitorid','total_purchases']]
ds_temp.groupby('total_purchases').size()

selecao = drugstore['total_purchases']==43
drugstore[selecao]

In [ ]:
# Store averages per basket size
ds_temp = ds_temp.groupby('order_transactionid').agg({'order_revenue': "mean"}).reset_index()
ds_temp = ds_temp[ds_temp.order_transactionid >= 3]

# Plot Relationship
plt.figure(figsize=(16,5))
ax = sns.regplot(data = ds_temp, 
                 x="order_revenue", 
                 y="order_transactionid")
plt.title("Relationship between Average Order & Revenue")
plt.xlabel("Order Value")
plt.ylabel("Revenue")
plt.show()

In [ ]:
# Create Summary Data
ds_temp = drugstore.groupby('visitor_visitorid').agg({'order_transactionid':'count'}).reset_index()
ds_temp = ds_temp.groupby('order_transactionid').agg({'visitor_visitorid':'count'}).reset_index()
ds_temp['pct_customers'] = ds_temp['visitor_visitorid'] / np.sum(ds_temp['visitor_visitorid']) * 100
ds_temp['transition'] = ds_temp.visitor_visitorid / ds_temp.visitor_visitorid.shift(2) * 100

# Plot relationship
ax = sns.set_style(style=None, rc=None )
fig, ax = plt.subplots(figsize=(25,10))
sns.lineplot(data = ds_temp['transition'], marker='o', sort = False, ax=ax)
sns.barplot(data = ds_temp, x='order_transactionid', y='pct_customers', alpha=0.5, color = 'green', ax=ax)
plt.title("Customers by Number of Orders and Transition Rate")
plt.xlabel("Number of Orders")
plt.ylabel("Percentage of Customers")
plt.show()

In [ ]:
# Clear Objects
del ds_temp, ax, fig

# Identify customers with multiple purchases
ds_multi = drugstore.groupby('visitor_visitorid').agg({'order_transactionid':'count'}).reset_index()
ds_multi = ds_multi[ds_multi['order_transactionid'] > 1]

# Filter for customers with multiple purchases
ds_interval = drugstore[drugstore.visitor_visitorid.isin(ds_multi.visitor_visitorid)]

del ds_multi

# Convert timestamp to date and order the dateframe by customer and order date
ds_interval = ds_interval[['visitor_visitorid','time_epoch_hr']]
ds_interval['date'] = pd.to_datetime(ds_interval['time_epoch_hr']).dt.date
ds_interval = ds_interval.sort_values(by =['visitor_visitorid','date'])

In [ ]:
# Calculate the time interval
ds_interval['previous_date'] = ds_interval.groupby('visitor_visitorid')['date'].shift(1)
ds_interval.dropna(inplace = True)
ds_interval['order_interval'] = (ds_interval['date'] - ds_interval['previous_date']).dt.days

# Remove orders made on the same day
df_interval = ds_interval[ds_interval['order_interval'] > 2]

# Plot interval
plt.figure(figsize =(25,10))
ax = sns.boxplot(df_interval['order_interval'])
plt.title("Distribution of Days Between Orders")
plt.xlabel("Days")
plt.show()

# Clear objects
del ax, df_interval 

In [ ]:
# Add order date to modelling dateframe
drugstore['date'] = pd.to_datetime(drugstore['time_epoch_hr']).dt.date

# Identify Max Purchase Date
v_maxdate = drugstore.date.max()

# Identify Max Purchase Date Per Customer
df_recency = drugstore.groupby('visitor_visitorid')['date'].max().reset_index()
df_recency['recency_days'] = (v_maxdate - df_recency['date']).dt.days
df_recency['segment_recency'] = np.where(df_recency['recency_days'] < 90, 'Active', 
                                         np.where(df_recency['recency_days'] <= 180, 'Inactive', 'Lapsed'))
# Plot
fig, ax = plt.subplots(1, 2, figsize=(18,8))
ax[0].set_title("Number of Customers")
ax[1].set_title("Distribution of Days")
sns.countplot(data = df_recency, x = 'segment_recency', order = ['Active', 'Inactive', 'Lapsed'], ax = ax[0])
sns.boxplot(data = df_recency, x = 'segment_recency', y = 'recency_days', order = ['Active', 'Inactive', 'Lapsed'], ax = ax[1])
plt.setp(ax[:], xlabel='Recency Segment')
plt.setp(ax[0], ylabel='Number of Customers')
plt.setp(ax[1], ylabel='Number of Days')
plt.show()

# Create Segment Dataframe
df_segments = df_recency[['visitor_visitorid', 'recency_days', 'segment_recency']].copy()

# Clear Object
del v_maxdate, df_recency, fig, ax